<a href="https://colab.research.google.com/github/arjangvt/CodeFolio/blob/main/ML/ANN/ANN_for_Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building a simple ANN using tensorflow.
__________________________________________________
<br>
<br>
<font size=2>
Written by: Arjang Fahim<br>
Date: 10-1-2021
Updated: 3-30-2022
</font>

# Step 1: Installation and setup

In [ ]:
'''
  Colab already installed tensorflow versions 1.x and 2.0,
  GPU and non GPU versions.
  For more information please check
  https://colab.research.google.com/notebooks/tensorflow_version.ipynb
'''

'\n  Colab already installed tensorflow versions 1.x and 2.0,\n  GPU and non GPU versions.\n  For more information please check\n  https://colab.research.google.com/notebooks/tensorflow_version.ipynb\n'

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Step 2: Data preprocessing

In this exercise we used "Churn Modeling" dataset from 
Kaggle.
The dataset can be found and downloded from <br>
https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling

After downloading the dataset unzip it and save it in a directory in your local machine.

In [ ]:
# importing the dataset

from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [ ]:
# use the print statement below to make sure the file is uploaded properly.
#print(uploaded)

In [ ]:
# chekcing the working directory

! ls

Churn_Modelling.csv  sample_data


In [ ]:
# reading the file to a dataframe

dataset = pd.read_csv('Churn_Modelling.csv')

In [ ]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# excluding the list of fields that are not used for learning
# the label field (Exited) also removed from the data
# the label is saved into another dataframe
# axis = 1 refers to column and axis = 0 refers to rows

x = dataset.drop(labels=['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis = 1)
y = dataset['Exited']

In [ ]:
x.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71
6,822,France,Male,50,7,0.00,2,1,1,10062.80
7,376,Germany,Female,29,4,115046.74,4,1,0,119346.88
8,501,France,Male,44,4,142051.07,2,0,1,74940.50
9,684,France,Male,27,2,134603.88,1,1,1,71725.73


In [ ]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [ ]:
# categorical (e.g Geography field) cannot be used directly.
# such fields need to be transformed to numerical format
# below we encode categorical columns to numbers

from sklearn.preprocessing import LabelEncoder

In [ ]:
label_1 = LabelEncoder()
x['Geography'] = label_1.fit_transform(x['Geography'])

In [ ]:
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,Female,42,2,0.00,1,1,1,101348.88
1,608,2,Female,41,1,83807.86,1,0,1,112542.58
2,502,0,Female,42,8,159660.80,3,1,0,113931.57
3,699,0,Female,39,1,0.00,2,0,0,93826.63
4,850,2,Female,43,2,125510.82,1,1,1,79084.10


In [ ]:
label_2 = LabelEncoder()
x['Gender'] = label_2.fit_transform(x['Gender'])

In [ ]:
x.head(8)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
5,645,2,1,44,8,113755.78,2,1,0,149756.71
6,822,0,1,50,7,0.00,2,1,1,10062.80
7,376,1,0,29,4,115046.74,4,1,0,119346.88


In [ ]:
# dummy trap refers to categorical variables that 
# 1- create multicollinearity 
# 2- the values cannot be compared. For example we cannot say germany > france
# to avoid this problem we do as follows

x = pd.get_dummies(x, drop_first=True, columns= ['Geography'])

In [ ]:
# Geography_1 and 2 are the binary representation of the column Geography

x.head(8)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
0,619,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,1
5,645,1,44,8,113755.78,2,1,0,149756.71,0,1
6,822,1,50,7,0.00,2,1,1,10062.80,0,0
7,376,0,29,4,115046.74,4,1,0,119346.88,1,0


In [ ]:
# splitting the dataset into train and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test =  train_test_split(x,y, test_size = 0.2, random_state=0)

In [ ]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
x_train

array([[ 0.16958176, -1.09168714, -0.46460796, ...,  1.10643166,
        -0.5698444 ,  1.74309049],
       [-2.30455945,  0.91601335,  0.30102557, ..., -0.74866447,
         1.75486502, -0.57369368],
       [-1.19119591, -1.09168714, -0.94312892, ...,  1.48533467,
        -0.5698444 , -0.57369368],
       ...,
       [ 0.9015152 ,  0.91601335, -0.36890377, ...,  1.41231994,
        -0.5698444 , -0.57369368],
       [-0.62420521, -1.09168714, -0.08179119, ...,  0.84432121,
        -0.5698444 ,  1.74309049],
       [-0.28401079, -1.09168714,  0.87525072, ...,  0.32472465,
         1.75486502, -0.57369368]])

In [ ]:
x_test

array([[-0.55204276, -1.09168714, -0.36890377, ...,  1.61085707,
         1.75486502, -0.57369368],
       [-1.31490297, -1.09168714,  0.10961719, ...,  0.49587037,
        -0.5698444 , -0.57369368],
       [ 0.57162971, -1.09168714,  0.30102557, ..., -0.42478674,
        -0.5698444 ,  1.74309049],
       ...,
       [-0.74791227,  0.91601335, -0.27319958, ...,  0.71888467,
        -0.5698444 ,  1.74309049],
       [-0.00566991,  0.91601335, -0.46460796, ..., -1.54507805,
         1.75486502, -0.57369368],
       [-0.79945688,  0.91601335, -0.84742473, ...,  1.61255917,
         1.75486502, -0.57369368]])

# Step 3: Building the Model

In [ ]:
# creating an object (Initialization of ANN)
model = tf.keras.models.Sequential()

In [ ]:
# typical way to calculate the number of units is (input_dim + 1)/2
# in this case 1+11 = 12 /2 = 6 

model.add(tf.keras.layers.Dense(units=6, activation = 'relu', input_dim=11))

In [ ]:
# adding second hidden layer
model.add(tf.keras.layers.Dense(units=6, activation = 'relu'))

In [ ]:
# outpu layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid' ))

In [ ]:
# compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


# Step 4: Training the model

In [ ]:
model.fit(x_train, y_train.to_numpy(), batch_size=10, epochs=20)

Epoch 1/20
800/800 [==============================] - 2s 2ms/step - loss: 0.5935 - accuracy: 0.7222
Epoch 2/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4575 - accuracy: 0.8034
Epoch 3/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4377 - accuracy: 0.8094
Epoch 4/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4275 - accuracy: 0.8133
Epoch 5/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4196 - accuracy: 0.8186
Epoch 6/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4113 - accuracy: 0.8267
Epoch 7/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4010 - accuracy: 0.8309
Epoch 8/20
800/800 [==============================] - 1s 2ms/step - loss: 0.3879 - accuracy: 0.8396
Epoch 9/20
800/800 [==============================] - 1s 2ms/step - loss: 0.3737 - accuracy: 0.8468
Epoch 10/20
800/800 [==============================] - 1s 2ms/step - loss: 0.3609 - accuracy: 0.8519

# Step 5: Model evaluation and prediction

In [ ]:
# model evaluation
print(type(y_test))
text_loss, test_acc = model.evaluate(x_test, y_test)

<class 'pandas.core.series.Series'>
63/63 [==============================] - 0s 2ms/step - loss: 0.3398 - accuracy: 0.8605


In [ ]:
print('Test Accuracy: {}'.format(test_acc) )

Test Accuracy: 0.8604999780654907


In [ ]:
y_pred = (model.predict(x_test) > 0.5).astype("int32")

In [ ]:
print(y_pred)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:
print(y_test)

9394    0
898     1
2398    0
5906    0
2343    0
       ..
1037    0
2899    0
9549    0
2740    0
6690    0
Name: Exited, Length: 2000, dtype: int64


In [ ]:
y_test = y_test.to_numpy()

In [ ]:
print(y_pred[11]) , print(y_test[11])

[0]
0


(None, None)

In [ ]:
# printing the "confusion matrix"
# diagnal elements are the correct predictions and non-diagonal 
# are wrong predictions

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
cm

array([[1523,   72],
       [ 207,  198]])

In [ ]:
acc_cm =  accuracy_score(y_test, y_pred)
print(acc_cm)

0.8605
